In [1]:
import pandas as pd

In [2]:
with open('./enz_flux_calculation.txt') as f:
    x = f.read().split('\n')
x = [i for i in x if i != '']

enzsyn_dict = dict(); enzload_dict = dict()
for i in x:
    enzid,v = i.split('\t')
    etype = enzid.split('-')[0]
    if etype == 'ENZSYN':
        if float(v) > 0:
            enzsyn_dict[enzid] = float(v)
    elif etype == 'ENZLOAD':
        if float(v) > 0:
            enzload_dict[enzid] = float(v)
    else:
        print('wtf')

In [ ]:
ptot = sum(enzsyn_dict.values())
enzsyn_frac_dict = {k:v/ptot for k,v in enzsyn_dict.items()}

In [3]:
#### Write enzyme load constraints based on enzyme flux calculated from proteome
eqn_list = []; kappvars = []; venzUB = [];

for k in enzload_dict.keys():
    id_ = k.split('-', maxsplit=1)[1]
    # Constraints
    lhs = "%mu% * v('RXN-" + id_ + "')"
    rhs = "venzLoadUB('ENZLOAD-" + id_ + "') * " + "kappvar('ENZLOAD-" + id_ + "')"
    rhs += " dv('" + i + "')"
    eqn_list.append(lhs + ' =l= ' + rhs + ';')
    
    # kapps as variables
    kappvars.append("'ENZLOAD-" + id_ + "'")
    
    # Enzyme flux upper bounds as constants
    venzUB.append("'ENZLOAD-" + id_ + "' " + str(enzload_dict[k]))
    
eqn_idx = ['EnzLoad'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzLoad'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

venzUB = ['/'] + venzUB + ['/']
kappvars = ['/'] + kappvars + ['/']

with open('./minFluxViolation/enzLoad_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./minFluxViolation/enzLoad_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
with open('./minFluxViolation/enzLoad_venzLoadUB.txt', 'w') as f:
    f.write('\n'.join(venzUB))
with open('./minFluxViolation/enzLoad_kappvarset.txt', 'w') as f:
    f.write('\n'.join(kappvars))

In [4]:
#### Write enzyme synthesis constraints based on enzyme flux calculated from proteome
eqn_list = []; venzUB = []

for k in enzsyn_dict.keys():
    # Constraints
    lhs = "v('" + k + "')"
    rhs = "venzSynUB('" + k + "')"
    eqn_list.append(lhs + ' =l= ' + rhs + ';')
    
    # Enzyme flux upper bounds as constants
    venzUB.append("'" + k + "' " + str(enzsyn_dict[k]))
    
eqn_idx = ['EnzSyn'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzSyn'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

venzUB = ['/'] + venzUB + ['/']

with open('./minFluxViolation/enzSyn_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./minFluxViolation/enzSyn_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))
with open('./minFluxViolation/enzSyn_venzSynUB.txt', 'w') as f:
    f.write('\n'.join(venzUB))

In [5]:
df_eqn = pd.read_excel('../scRBA_build_GAMS_model/model/RBA_stoichiometry.xlsx')
df_eqn.index = df_eqn.id.to_list()
idx = df_eqn[df_eqn.id.str.contains('^ENZLOAD-')].index

In [6]:
#### Add slack variables to reactions whose enzyme concentrations are zeroes or not presence in data
eqn_list = []; venzUB = []

for i in idx:
    if i not in enzload_dict.keys():
        # Constraints
        lhs = "v('" + i + "')"
        rhs = "dv('" + i + "')"
        eqn_list.append(lhs + ' =l= ' + rhs + ';')

eqn_idx = ['EnzFree'+str(i) for i in range(0, len(eqn_list))]
eqn_list = ['EnzFree'+str(i)+'.. ' + eqn_list[i] for i in range(0, len(eqn_list))]

with open('./minFluxViolation/enzFree_declares.txt', 'w') as f:
    f.write('\n'.join(eqn_idx))
with open('./minFluxViolation/enzFree_eqns.txt', 'w') as f:
    f.write('\n'.join(eqn_list))

In [7]:
df_enz = pd.read_excel('../scRBA_build_GAMS_model/input/ENZYME_stoich_curation_2021-10-01.xlsx')
df_enz.index = df_enz.id.to_list()

mw_dict = dict()
for i in df_enz.index:
    enzid = 'ENZSYN-' + i.split('-', maxsplit=2)[2]
    mw_dict[enzid] = df_enz.loc[i, 'MW (g/mmol)']

In [28]:
enztot = 0
for k,v in enzsyn_dict.items():
    enztot += v * mw_dict[k]
print(enztot / 0.4)

0.2167681731914843
